In [1]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/habkaffee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/habkaffee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
dataset = pd.read_csv('data/DATASET.csv')
dataset

,Review,label
0,"Great music service, the audio is high quality...",POSITIVE
1,Please ignore previous negative rating. This a...,POSITIVE
2,"This pop-up ""Get the best Spotify experience o...",NEGATIVE
3,Really buggy and terrible to use as of recently,NEGATIVE
4,Dear Spotify why do I get songs that I didn't ...,NEGATIVE
...,...,...
52697,Yes the best,POSITIVE
52698,Spotify won my heart in Feb 2024 You won my he...,POSITIVE
52699,I tried to open the app and it wont open I res...,POSITIVE
52700,Good,POSITIVE


# Preprocessing

In [3]:
exclude_symbols = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                            '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                            '\x7f', '\xad', '¤', '\xa0', '\u200b', '–']) + string.punctuation
regex_symb = re.compile('[%s]' % re.escape(exclude_symbols))

In [4]:
stop_words = stopwords.words('english')
pattern = r'\b(?:' + '|'.join(stop_words) + r')\b'
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
def lemmatize_sentence(sentence):
    if isinstance(sentence, str):
      sentence = ' '.join([lemmatizer.lemmatize(word) for word in sentence.split() if word not in string.digits])
    return sentence

In [7]:
dataset['Review'] = dataset['Review'].str.lower()
dataset['label'] = dataset['label'].str.replace('POSITIVE', '1').replace('NEGATIVE', '0')
dataset['Review'] = dataset['Review'].str.replace(pattern, '', regex=True)
dataset['Review'] = dataset['Review'].str.replace(f'[{string.punctuation}]+', ' ', regex=True)
dataset['Review'] = dataset['Review'].str.replace(f' +', ' ', regex=True)
dataset['Review'] = dataset['Review'].str.replace(f'[0-9]+', '', )
dataset['Review'] = dataset['Review'].apply(lemmatize_sentence)
dataset = dataset[dataset['Review'].apply(lambda x : isinstance(x, str) and x.isascii())]
dataset

,Review,label
0,great music service audio high quality app eas...,1
1,please ignore previous negative rating app sup...,1
2,pop get best spotify experience android annoyi...,0
3,really buggy terrible use recently,0
4,dear spotify get song put playlist shuffle play,0
...,...,...
52697,yes best,1
52698,spotify heart feb 2024 heart music lyric langu...,1
52699,tried open app wont open restarted phone ill t...,1
52700,good,1


In [8]:
all_words = set()
for data in dataset['Review']:
    words = data.split()
    all_words.update(words)
all_words

{'unexpected',
 'nya',
 'continue',
 'musin',
 'curated',
 'listener',
 'recognized',
 'lost',
 'specified',
 'speck',
 'outraging',
 'updat',
 'lotta',
 'memorize',
 'reminiscent',
 '1000kms',
 'someting',
 'viceversa',
 'repertoire',
 'collabs',
 '40ish',
 'whacky',
 'spotiflyer',
 'allonnee',
 'format',
 'bubby',
 'show',
 'block',
 'response',
 'marino',
 'netbanking',
 'apni',
 'g',
 'fed',
 'slacker',
 'aaaa',
 'incompetence',
 'enhance',
 'worked',
 'funk',
 'beech',
 'mainstream',
 'comforting',
 'humiliate',
 'rn',
 'laggs',
 'flattening',
 'goodddddddddddddddddddddddddddddddddddd',
 'horrble',
 'versiin',
 'thanyou',
 'rhetoric',
 'sandesh',
 'sri',
 'somuchhhh',
 'contend',
 'command',
 'pig',
 'wildest',
 'crushinv',
 'tv',
 'rectified',
 'caters',
 'west',
 'useage',
 '1000000',
 'gjb',
 'variedad',
 'book',
 'begs',
 'treatment',
 'youtubers',
 'arrogance',
 'loop',
 'performing',
 'wagon',
 'invaluable',
 'reordered',
 'seize',
 'unsync',
 'goodu',
 'pronab',
 'killer',


In [ ]:
cnt_vectorizer = CountVectorizer()